In [1]:
import pandas as pd
import requests
from tqdm import tqdm


In [4]:
ruta_archivo = "BASE DE DATOS ECOHENO.xlsx"
df = pd.read_excel(ruta_archivo)
df.head()

,AÑO,MES,DÍAS,LABORAL,TEMPORADA,CULTIVO,RECOLECCION,HENIFICACION,FECHA COMPLETA,ACTIVIDAD DEL DÍA,HA,NOTAS
0,2022,Enero,3,Si,Verano,Si,Si,Si,2022-01-03,CORTE,1.0,Se corta 1 ha completa con tractor
1,2022,Enero,4,Si,Verano,Si,Si,Si,2022-01-04,SECADO – Día 1,1.0,Sol fuerte
2,2022,Enero,5,Si,Verano,Si,Si,Si,2022-01-05,SECADO – Día 2,1.0,Sol fuerte
3,2022,Enero,6,Si,Verano,Si,Si,Si,2022-01-06,VOLTEO – Día 1,1.0,Volteo manual
4,2022,Enero,7,Si,Verano,Si,Si,Si,2022-01-07,VOLTEO – Día 2,1.0,Finalizar


In [5]:
# Coordenadas oficiales del municipio de Sopetrán, Antioquia
latitud = 6.4998
longitud = -75.7460

print("Coordenadas cargadas:")
print(f"Latitud: {latitud}")
print(f"Longitud: {longitud}")

Coordenadas cargadas:
Latitud: 6.4998
Longitud: -75.746


In [7]:
df.columns

Index(['AÑO', 'MES', 'DÍAS', 'LABORAL', 'TEMPORADA', 'CULTIVO', 'RECOLECCION',
       'HENIFICACION', 'FECHA COMPLETA', 'ACTIVIDAD DEL DÍA', 'HA', 'NOTAS'],
      dtype='object')

In [8]:
# === BLOQUE 4: Construir fecha y definir función de clima ===

import pandas as pd
import requests

# Convertimos MES a número
meses = {
    "Enero": 1, "Febrero": 2, "Marzo": 3, "Abril": 4,
    "Mayo": 5, "Junio": 6, "Julio": 7, "Agosto": 8,
    "Septiembre": 9, "Octubre": 10, "Noviembre": 11, "Diciembre": 12
}

# Crear columna FECHA en formato YYYY-MM-DD
df["FECHA"] = pd.to_datetime(
    df["AÑO"].astype(str) + "-" +
    df["MES"].map(meses).astype(str) + "-" +
    df["DÍAS"].astype(str)
)

# Función de clima
def obtener_clima(fecha, lat, lon):
    fecha_str = fecha.strftime("%Y-%m-%d")
    url = (
        "https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}"
        f"&start_date={fecha_str}&end_date={fecha_str}"
        "&daily=temperature_2m_max,temperature_2m_min,precipitation_sum"
        "&timezone=America/Bogota"
    )

    r = requests.get(url).json()

    try:
        return {
            "temp_max": r["daily"]["temperature_2m_max"][0],
            "temp_min": r["daily"]["temperature_2m_min"][0],
            "lluvia_mm": r["daily"]["precipitation_sum"][0]
        }
    except:
        return {"temp_max": None, "temp_min": None, "lluvia_mm": None}


In [9]:
# === BLOQUE 5: Agregar clima histórico a la base ===

from tqdm import tqdm

# Nuevas columnas vacías
df["TEMP_MAX"] = None
df["TEMP_MIN"] = None
df["LLUVIA_MM"] = None

# Aplicamos el proceso fila por fila
for i in tqdm(df.index, desc="Procesando clima"):
    fecha = df.loc[i, "FECHA"]        # Fecha ya construida en el bloque anterior
    clima = obtener_clima(fecha, latitud, longitud)
    
    df.loc[i, "TEMP_MAX"] = clima["temp_max"]
    df.loc[i, "TEMP_MIN"] = clima["temp_min"]
    df.loc[i, "LLUVIA_MM"] = clima["lluvia_mm"]


Procesando clima: 100%|██████████| 1154/1154 [17:00<00:00,  1.13it/s]


In [11]:
# Inicializar columna
df["DOR"] = 1  # Asumimos operativo, luego aplicamos exclusiones


# --- Reglas generales de exclusión ---
df.loc[df["LLUVIA_MM"] > 12, "DOR"] = 0
df.loc[df["TEMP_MAX"] > 38.5, "DOR"] = 0


# --- Reglas específicas por actividad ---
def ajustar_por_actividad(row):
    actividad = str(row["ACTIVIDAD DEL DÍA"]).upper()
    lluvia = row["LLUVIA_MM"]

    # Corte
    if "CORTE" in actividad:
        if lluvia > 8:
            return 0

    # Secado
    if "SECADO" in actividad:
        if lluvia > 2:
            return 0

    # Volteo
    if "VOLTEO" in actividad:
        if lluvia > 5:
            return 0

    # Henificación (armado de pacas)
    if "HENIFIC" in actividad:
        if lluvia > 0:
            return 0

    # Recolección final
    if "RECOLEC" in actividad:
        if lluvia > 3:
            return 0

    return row["DOR"]


df["DOR"] = df.apply(ajustar_por_actividad, axis=1)

In [12]:
nombre_salida = "BASE_ECOHENO_CLIMA.xlsx"
df.to_excel(nombre_salida, index=False)
print("Archivo guardado correctamente como:", nombre_salida)

Archivo guardado correctamente como: BASE_ECOHENO_CLIMA.xlsx
